Artık Ağlar Uygulaması

In [2]:
import tensorflow as tf 

from kerastuner import HyperModel, RandomSearch

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.utils import to_categorical



# data loading
(train_imgs, train_labels), (test_imgs, test_labels) = fashion_mnist.load_data()

# data preprocessing
train_images = train_imgs.reshape(-1, 28, 28, 1) / 255.0  # normalde "60000" yazmam gerekiyordu ama "-1" diyerek kendisi seçmesi için bıraktım.
test_images = test_imgs.reshape(-1, 28, 28, 1) / 255.0

# one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

C:\Users\mrtmm\AppData\Local\Temp\ipykernel_9392\4025274611.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel, RandomSearch


In [ ]:
def residual_block(x, filters, kernel_size=3, stride =1):
    shortcut = x
    x = Conv2D(filters, kernel_size=kernel_size,strides =stride, padding='same' )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, kernel_size=kernel_size, strides = stride, padding='same')(x)
    x = BatchNormalization()(x)
    # eğer girişten gelen verinin boyutu filtre sayısına eşit değilse
    if shortcut.shape[-1] != filters :
        # giriş verisinin boyutunu eşitlemek için 1x1 konvolusyon uygulayalım
        shortcut = Conv2D(filters, kernel_size=1, strides = stride,padding='same')(shortcut)

    # residual bağlantı: giriş verisi ile çıkış verisini toplayalım
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

In [4]:
class ResNetModel(HyperModel):
    def build(self, hp):
        inputs = Input(shape=(28,28,1))

        x = Conv2D(filters = hp.Int("initial_filters", min_value=32, max_value=256, step=32)
                   , kernel_size=3, strides=1, padding='same')(inputs)
        x = BatchNormalization()(x)

        #residual blok ekleyelim
        for i in range(hp.Int('num_blocks',min_value = 1,max_value =3, step = 1)):
            x = residual_block(x, hp.Int("res_filters_" + str(i), min_value=32, max_value=128, step=32))

        # sınıflandırma katmanı
        x = Flatten()(x)
        z = Dense(128, activation = "relu")(x)
        outputs = Dense(10, activation = "softmax")(z)

        model = Model(inputs, outputs)

        model.compile(optimizer = Adam(hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")),
                      loss = "categorical_crossentropy",
                      metrics = ["accuracy"])
        return model
ResNetModel()

In [5]:
tuner = RandomSearch(
    ResNetModel(),
    objective = "val_accuracy", # tuning referans değeri
    max_trials = 2, # toplam deneme sayısı
    executions_per_trial = 1, # her denemede kaç kere eğitim yapılacağı 
    directory = "my_dir",
    project_name = "fashion_mnist"
)

tuner.search(train_images, train_labels, epochs = 1, validation_data = (test_images, test_labels))

# en iyi modeli alalım
best_model = tuner.get_best_models(num_models=1)[0]

# test seti ile en iyi modeli test edelim 
test_loss, test_acc = best_model.evaluate(test_images, test_labels)
print(test_acc)
print(test_loss)

Reloading Tuner from my_dir\fashion_mnist\tuner0.json



c:\Users\mrtmm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 58 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 118ms/step - accuracy: 0.8946 - loss: 0.2852
0.896399974822998
0.28106194734573364
